# base_dictionary.py

In [13]:
from dictionary.word_frequency import WordFrequency


# -------------------------------------------------
# Base class for dictionary implementations. DON'T CHANGE THIS FILE.
#
# __author__ = 'Son Hoang Dau'
# __copyright__ = 'Copyright 2022, RMIT University'
# -------------------------------------------------

class BaseDictionary:
    def build_dictionary(self, words_frequencies: [WordFrequency]):
        """
        construct the data structure to store nodes
        @param words_frequencies: list of (word, frequency) to be stored
        """
        pass

    def search(self, word: str) -> int:
        """
        search for a word
        @param word: the word to be searched
        @return: frequency > 0 if found and 0 if NOT found
        """
        pass

    def add_word_frequency(self, word_frequency: WordFrequency) -> bool:
        """
        add a word and its frequency to the dictionary
        @param word_frequency: (word, frequency) to be added
        @return: True whether succeeded, False when word is already in the dictionary
        """
        pass

    def delete_word(self, word: str) -> bool:
        """
        delete a word from the dictionary
        @param word: word to be deleted
        @return: whether succeeded, e.g. return False when point not found
        """
        pass

    def autocomplete(self, prefix_word: str) -> [WordFrequency]:
        """
        return a list of 3 most-frequent words in the dictionary that have 'prefix_word' as a prefix
        @param prefix_word: word to be autocompleted
        @return: a list (could be empty) of (at most) 3 most-frequent words with prefix 'prefix_word'
        """
        pass




# list_dictionary.py

In [14]:
from dictionary.word_frequency import WordFrequency
from dictionary.base_dictionary import BaseDictionary
import time
import math

# ------------------------------------------------------------------------
# This class is required TO BE IMPLEMENTED. List-based dictionary implementation.
#
# __author__ = 'Son Hoang Dau'
# __copyright__ = 'Copyright 2022, RMIT University'
# ------------------------------------------------------------------------

class ListDictionary(BaseDictionary):
    def __init__(self):
        self.data: list = None;

    def partition(self, data, i, k, by):
        midpoint = i + (k - i) // 2
        pivot = getattr(data[midpoint], by)

        done = False
        l = i
        h = k
        while not done:
            while getattr(data[l], by) < pivot:
                l = l + 1
            while pivot < getattr(data[h], by):
                h = h - 1
            if l >= h:
                done = True
            else:
                temp = data[l]
                data[l] = data[h]
                data[h] = temp
                l = l + 1
                h = h - 1
        return h

    def quicksort(self, data, i, k, by: str):
        j = 0
        if i >= k:
            return
        j = self.partition(data, i, k, by)
        self.quicksort(data, i, j, by)
        self.quicksort(data, j + 1, k, by)
        return

    # merge sort algorithm sourced from Zybooks
    def merge(self, data, i, j, k, by: str):
        merged_size = k - i + 1  # Size of merged partition
        merged_data = [0] * merged_size  # Dynamically allocates temporary array
        # for merged numbers
        merge_pos = 0  # Position to insert merged number
        left_pos = i  # Initialize left partition position
        right_pos = j + 1  # Initialize right partition position

        # Add smallest element from left or right partition to merged numbers
        while left_pos <= j and right_pos <= k:
            if getattr(data[left_pos], by) <= getattr(data[right_pos], by):
                merged_data[merge_pos] = data[left_pos]
                left_pos += 1
            else:
                merged_data[merge_pos] = data[right_pos]
                right_pos += 1
            merge_pos = merge_pos + 1

        # If left partition is not empty, add remaining elements to merged numbers
        while left_pos <= j:
            merged_data[merge_pos] = data[left_pos]
            left_pos += 1
            merge_pos += 1

        # If right partition is not empty, add remaining elements to merged numbers
        while right_pos <= k:
            merged_data[merge_pos] = data[right_pos]
            right_pos = right_pos + 1
            merge_pos = merge_pos + 1

        # Copy merge number back to numbers
        for merge_pos in range(merged_size):
            data[i + merge_pos] = merged_data[merge_pos]

    def merge_sort(self, data, i, k, by: str):
        j = 0

        if i < k:
            j = (i + k) // 2  # Find the midpoint in the partition

            # Recursively sort left and right partitions
            self.merge_sort(data, i, j, by)
            self.merge_sort(data, j + 1, k, by)

            # Merge left and right partition in sorted order
            self.merge(data, i, j, k, by)

    def __str__(self):
        str = ""
        for items in self.data:
            str += f"({items.word}, {items.frequency})\n"
        return str

    def build_dictionary(self, words_frequencies: [WordFrequency]):
        """
        construct the data structure to store nodes
        @param words_frequencies: list of (word, frequency) to be stored
        """
        self.data = words_frequencies
        # self.quicksort(self.data, 0, len(self.data) - 1, "word")
        self.data.sort(key=lambda x: x.word)

    def binSearch(self, word:str) -> (bool, int):
        """
        binary search for a word
        @param word: the word to be searched
        @return: (True, the index of word_frequencies) OR (False, the index of word_frequencies to be inserted into)
        """
        low, mid, high = 0, 0, len(self.data) - 1

        while low <= high:
            mid = (low + high) // 2
            if self.data[mid].word > word:
                high = mid - 1
            elif self.data[mid].word < word:
                low = mid + 1
            else:
                return (True, mid)
        if word >= self.data[mid].word:
            return (False, mid + 1)
        else:
            return (False, mid)

    def binSearchAC(self, prefix_word:str) -> int:
        """
        binary search for a prefix
        @param prefix: the prefix to be searched
        @return: if found: the index of the first encountered word with the same prefix; if not: -1
        """
        # The implementation is almost identical to binSearch except that prefix is compared to word upto its own length
        low, mid, high = 0, 0, len(self.data) - 1

        while low <= high:
            mid = (low + high) // 2
            if self.data[mid].word[:len(prefix_word)] > prefix_word:
                high = mid - 1
            elif self.data[mid].word[:len(prefix_word)] < prefix_word:
                low = mid + 1
            else:
                return mid
        return -1

    def getAutocompleteList(self, prefix_word: str, idx: int) -> [WordFrequency]:
        """
        add all the words sharing the same prefix_word to a list and return it unsorted
        @param prefix_word: the prefix_word to be searched, idx: the starting index to search from in both directions (left and right)
        @return: an unsorted list containing all the words sharing the same prefix_word
        """
        res = []
        # Add the first word
        res.append(self.data[idx])
        left_idx = idx - 1
        right_idx = idx + 1

        if left_idx >= 0:
            curr_left_word = self.data[left_idx].word[:len(prefix_word)]
        # Add words to the left of the first word
        while left_idx >= 0 and curr_left_word == prefix_word:
            res.append(self.data[left_idx])
            left_idx -= 1
            curr_left_word = self.data[left_idx].word[:len(prefix_word)]

        if right_idx <= len(self.data) - 1:
            curr_right_word = self.data[right_idx].word[:len(prefix_word)]
        # Add words to the right of the first word
        while right_idx <= len(self.data) - 1 and curr_right_word == prefix_word:
            res.append(self.data[right_idx])
            right_idx += 1
            curr_right_word = self.data[right_idx].word[:len(prefix_word)]

        return res

    def search(self, word: str) -> int:
        """
        search for a word
        @param word: the word to be searched
        @return: frequency > 0 if found and 0 if NOT found
        """
        # Employ binary search
        isFound, foundIdx = self.binSearch(word)
        if not isFound:
            return 0
        else:
            return self.data[foundIdx].frequency

    def add_word_frequency(self, word_frequency: WordFrequency) -> bool:
        """
        add a word and its frequency to the dictionary
        @param word_frequency: (word, frequency) to be added
        :return: True whether succeeded, False when word is already in the dictionary
        """
        # Employ binary search
        word = word_frequency.word
        isFound, foundIdx = self.binSearch(word)
        actualLength = len(self.data)
        if isFound:
            return False
        # If not found, add the word in self.data
        else:
            # Create space to shuffle elements to the right by 1
            self.data.append(None)
            for i in range(actualLength - 1, foundIdx - 1, -1):
                self.data[i + 1] = self.data[i]
            self.data[foundIdx] = word_frequency
            return True

    def delete_word(self, word: str) -> bool:
        """
        delete a word from the dictionary
        @param word: word to be deleted
        @return: whether succeeded, e.g. return False when point not found
        """
        isFound, foundIdx = self.binSearch(word)
        if isFound:
            for i in range(foundIdx, len(self.data) - 1):
                self.data[i] = self.data[i + 1]
            # In all cases, the last element will be deleted if the word is found
            del self.data[-1]
            return True
        # If found, delete the word in self.data
        else:
            return False



    def autocomplete(self, prefix_word: str) -> [WordFrequency]:
        """
        return a list of 3 most-frequent words in the dictionary that have 'prefix_word' as a prefix
        @param prefix_word: word to be autocompleted
        @return: a list (could be empty) of (at most) 3 most-frequent words with prefix 'prefix_word'
        """
        # As soon as prefix_word matches with any word, scan all the words to its left and right and put them in a new list
        # Iterate them only once to find the 3 most-frequent words
        idx = self.binSearchAC(prefix_word)
        if idx == -1:
            return []
        else:
            lst = self.getAutocompleteList(prefix_word, idx)
            self.merge_sort(lst, 0, len(lst) - 1, "frequency")
            return lst[-3:][::-1]


# hashtable_dictionary.py

In [15]:
from dictionary.base_dictionary import BaseDictionary
from dictionary.word_frequency import WordFrequency

# ------------------------------------------------------------------------
# This class is required TO BE IMPLEMENTED. Hash-table-based dictionary.
#
# __author__ = 'Son Hoang Dau'
# __copyright__ = 'Copyright 2022, RMIT University'
# ------------------------------------------------------------------------

class HashTableDictionary(BaseDictionary):
    def __init__(self):
        self.data = {}

    def build_dictionary(self, words_frequencies: [WordFrequency]):
        """
        construct the data structure to store nodes
        @param words_frequencies: list of (word, frequency) to be stored
        """
        self.data = {entry.word: entry.frequency for entry in words_frequencies}

    def search(self, word: str) -> int:
        """
        search for a word
        @param word: the word to be searched
        @return: frequency > 0 if found and 0 if NOT found
        """
        return self.data.get(word, 0)

    def add_word_frequency(self, word_frequency: WordFrequency) -> bool:
        """
        add a word and its frequency to the dictionary
        @param word_frequency: (word, frequency) to be added
        :return: True whether succeeded, False when word is already in the dictionary
        """
        freq = self.search(word_frequency.word)
        if freq > 0:
            return False
        else:
            self.data[word_frequency.word] = word_frequency.frequency
            return True

    def delete_word(self, word: str) -> bool:
        """
        delete a word from the dictionary
        @param word: word to be deleted
        @return: whether succeeded, e.g. return False when point not found
        """
        freq = self.search(word)
        if freq > 0:
            del self.data[word]
            return True
        else:
            return False


    def autocomplete(self, word: str) -> [WordFrequency]:
        """
        return a list of 3 most-frequent words in the dictionary that have 'word' as a prefix
        @param word: word to be autocompleted
        @return: a list (could be empty) of (at most) 3 most-frequent words with prefix 'word'
        """
        # Find the keys that start with a given prefix
        autoDic = {key: freq for key, freq in self.data.items() if key.startswith(word)}
        # Use Python's built-in sorting algorithm to sort autoDic by frequency and return the last three elements in descending order.
        return [WordFrequency(key, freq) for key, freq in sorted(autoDic.items(), key=lambda item: item[1])][-3:][::-1]


# tearnarysearchtree_dictionary.py

In [16]:
from dictionary.base_dictionary import BaseDictionary
from dictionary.word_frequency import WordFrequency
from dictionary.node import Node


# ------------------------------------------------------------------------
# This class is required to be implemented. Ternary Search Tree implementation.
#
# __author__ = 'Son Hoang Dau'
# __copyright__ = 'Copyright 2022, RMIT University'
# ------------------------------------------------------------------------


class TernarySearchTreeDictionary(BaseDictionary):
    def __init__(self):
        self.root = Node()

    def build_dictionary(self, words_frequencies: [WordFrequency]):
        """
        construct the data structure to store nodes
        @param words_frequencies: list of (word, frequency) to be stored
        """
        for idx, entry in enumerate(words_frequencies):
            self.add_word_frequency(entry)

    def search(self, word: str) -> int:
        """
        search for a word
        @param word: the word to be searched
        @return: frequency > 0 if found and 0 if NOT found
        """
        endNode = self.search_from_node(self.root, word, 0)
        if endNode == None:
            return 0
        elif endNode.end_word == False:
            return 0
        else:
            return endNode.frequency



    def search_from_node(self, currNode, word, currIdx):
        """
        search for a word recursively
        @param node, word, currIdx: node to start from, the word to be searched, currIdx to search curLetter at
        @return: frequency > 0 if found and 0 if NOT found
        """
        currLetter = word[currIdx]
        if currNode == None or currNode.letter == None:
            return None
        if currNode.letter < currLetter:
            return self.search_from_node(currNode.right, word, currIdx)
        elif currNode.letter > currLetter:
            return self.search_from_node(currNode.left, word, currIdx)
        elif currIdx < len(word) - 1:
            return self.search_from_node(currNode.middle, word, currIdx + 1)
        else:
            return currNode


    def add_word_frequency(self, word_frequency: WordFrequency) -> bool:
        """
        add a word and its frequency to the dictionary
        @param word_frequency: (word, frequency) to be added
        :return: True whether succeeded, False when word is already in the dictionary
        """
        return self.add_word_from_root(self.root, word_frequency.word, word_frequency.frequency, 0)

    def add_word_from_root(self, currNode, word, freq, currIdx) -> bool:
        """
        add a word recursively
        @param currNode, word, freq, currIdx: currNode initially self.root; currIdx is used for the base case.
        :return: True if addition is successful, false if the word being added already exists.
        """
        currLetter = word[currIdx]
        # Base case on the last word
        if currIdx == len(word) - 1:
            if currNode.letter == None:
                currNode.letter = currLetter
                currNode.frequency = freq
                currNode.end_word = True
                return True
            # If currNode is the same as curLetter
            else:
                if currNode.letter > currLetter:
                    if currNode.left == None:
                        currNode.left = Node()
                    currNode = currNode.left
                    return self.add_word_from_root(currNode, word, freq, currIdx)
                elif currNode.letter < currLetter:
                    if currNode.right == None:
                        currNode.right = Node()
                    currNode = currNode.right
                    return self.add_word_from_root(currNode, word, freq, currIdx)
                else:
                    if currNode.end_word == True:
                        return False
                    else:
                        currNode.frequency = freq
                        currNode.end_word = True
                        return True
        # Recursive case
        else:
            # When no word is present
            if currNode.letter == None:
                currNode.letter = currLetter
                currNode.middle = Node()
                currNode = currNode.middle
                return self.add_word_from_root(currNode, word, freq, currIdx + 1)
            elif currNode.letter < currLetter:
                if currNode.right == None:
                    currNode.right = Node()
                currNode = currNode.right
                return self.add_word_from_root(currNode, word, freq, currIdx)
            elif currNode.letter > currLetter:
                if currNode.left == None:
                    currNode.left = Node()
                currNode = currNode.left
                return self.add_word_from_root(currNode, word, freq, currIdx)
            else:
                if currNode.middle == None:
                    currNode.middle = Node()
                currNode = currNode.middle
                return self.add_word_from_root(currNode, word, freq, currIdx + 1)

    def delete_word(self, word: str) -> bool:
        """
        delete a word from the dictionary
        @param word: word to be deleted
        @return: whether succeeded, e.g. return False when point not found
        """
        # First, search for a word
        # endNode = self.search_from_node(self.root, word, 0)
        # if not endNode or endNode.end_word == False:
        #     return False
        # else:
        #     endNode.frequency = None
        #     endNode.end_word = False
        #     if endNode.left == None and endNode.middle == None and endNode.right == None:
        #         del endNode
        # return True
        deleteStatus = [False]
        self.delete_from_node(self.root, word, 0, deleteStatus)
        return deleteStatus[0]

    def delete_from_node(self, currNode, word, currIdx, deleteStatus):
        """
        delete a word recursively
        @param prevNode, currNode, word, currIdx
        @return: False if not found or end_word equals False, True if found and end_word equals True
        """
        currLetter = word[currIdx]
        if currNode == None or currNode.letter == None:
            return False
        if currNode.letter < currLetter:
            if self.delete_from_node(currNode.right, word, currIdx, deleteStatus):
                currNode.right = None
            else:
                return False
        elif currNode.letter > currLetter:
            if self.delete_from_node(currNode.left, word, currIdx, deleteStatus):
                currNode.left = None
            else:
                return False
        elif currIdx < len(word) - 1:
            if self.delete_from_node(currNode.middle, word, currIdx + 1, deleteStatus):
                currNode.middle = None
            else:
                return False
        else:
            if currNode.end_word:
                deleteStatus[0] = True
                currNode.frequency = None
                currNode.end_word = False

        if currNode.end_word == False:
            if currNode.left == None and currNode.middle == None and currNode.right == None:
                return True
            else:
                return False
        else:
            return False

    def add_ac_words(self, currNode: Node, compoundWord: str, ac_lst: list) -> [WordFrequency]:
        """
        Recursively traverse all the children nodes of currNode and create an instance of WordFrequency
        using compoundWord and the frequency of currNode if its end_word is True.
        @param currNode, compoundWord, ac_lst: compoundWord to keep track of the word to be added
        ac_lst: the list to which an instance of WordFrequency is added
        @return: a list (could be empty) of all the words with prefix 'word'
        """

        # Base Case 1: currNode is None
        if currNode == None:
            return
        # Recursive case:
        else:
            if currNode.end_word == True:
                ac_lst.append(WordFrequency(compoundWord + currNode.letter, currNode.frequency))
            self.add_ac_words(currNode.left, compoundWord, ac_lst)
            self.add_ac_words(currNode.middle, compoundWord + currNode.letter, ac_lst)
            self.add_ac_words(currNode.right, compoundWord, ac_lst)

    def autocomplete(self, word: str) -> [WordFrequency]:
        """
        return a list of 3 most-frequent words in the dictionary that have 'word' as a prefix
        @param word: word to be autocompleted
        @return: a list (could be empty) of (at most) 3 most-frequent words with prefix 'word'
        """
        # a list of words to be autocompleted
        ac_lst = []

        # Find the prefix
        currNode = self.search_from_node(self.root, word, 0)

        # If the prefix does not exist
        if not currNode:
            return ac_lst
        else:
            # If the currNode's end_word is true
            if currNode.end_word == True:
                ac_lst.append(WordFrequency(word, currNode.frequency))
            self.add_ac_words(currNode.middle, word, ac_lst)

            # Python's built-in Timsort
            ac_lst.sort(key=lambda wordFrequency: wordFrequency.frequency, reverse=True)

        return ac_lst[:3]


# word_frequency.py

In [17]:

# -------------------------------------------------
# __author__ = 'Son Hoang Dau'
# __copyright__ = 'Copyright 2022, RMIT University'
# -------------------------------------------------

# Class representing a word and its frequency
class WordFrequency:
    def __init__(self, word: str, frequency: int):
        self.word = word
        self.frequency = frequency


# node.py

In [18]:

# -------------------------------------------------
# __author__ = 'Son Hoang Dau'
# __copyright__ = 'Copyright 2022, RMIT University'
# -------------------------------------------------

# DON'T CHANGE THIS FILE
# Class representing a node in the Ternary Search Tree
class Node:

    def __init__(self, letter=None, frequency=None, end_word=False):
        self.letter = letter            # letter stored at this node
        self.frequency = frequency      # frequency of the word if this letter is the end of a word
        self.end_word = end_word        # True if this letter is the end of a word
        self.left = None    # pointing to the left child Node, which holds a letter < self.letter
        self.middle = None  # pointing to the middle child Node
        self.right = None   # pointing to the right child Node, which holds a letter > self.letter


## General plan for the Empirical Analysis of Algorithm Time Efficiency

1. Understand the experiment's purpose
    - to find which data structure is most appropriate for given scenarios.
2. Decide on the efficiency metric M to be measured and the measurement unit
    - We're required to measure a time unit in seconds
3. Decide on characteristics of the input sample
    - (500, 1000, 2000, 4000, 8000, 16,000, 32,000, 64,000, 128,000)
4. Prepare a program implementing the algorithm (or algorithms) for the experimentation.                         
5. Generate a sample of inputs.
6. Run the algorithm (or algorithms) on the sample's inputs and record the data observed.
7. Anaylse the data obtained.


### Generate a sample of inputs

In [7]:
import random

input_sizes = [500, 1000, 2000, 4000, 8000, 16000, 32000, 64000, 128000]
num_datasets_per_input = 10

for i in range(len(input_sizes)):
    with open("sampleData200k.txt", 'r') as f:
        lines = f.readlines()
        for j in range(num_datasets_per_input):
            # Shuffle the list of lines in place
            random.shuffle(lines)
            
            # Generate the main datasets to build dictionaries
            with open("generatedData/" + str(input_sizes[i]) + '-' + str(j) + '.txt', 'w') as f_dic: # 50-0.txt
                f_dic.writelines(lines[0:input_sizes[i]])
            # Generate the command lines to add words for Scenario 1
            with open("generatedData/" + str(input_sizes[i]) + '-' + str(j) + '-add.in', 'w' ) as f_add:
                lst_words_to_add = lines[input_sizes[i] : input_sizes[i] + input_sizes[i] // 2]
                command_lines = ['A ' + word for word in lst_words_to_add]
                f_add.writelines(command_lines)
            # Generate the command lines to delete words for Scenario 2
            with open("generatedData/" + str(input_sizes[i]) + '-' + str(j) + '-del.in', 'w') as f_del:
                lst_words_in_dic = lines[0:input_sizes[i]]
                random.shuffle(lst_words_in_dic)
                lst_words_to_del = lst_words_in_dic[0: input_sizes[i] // 2]
                command_lines = ['D ' +  word for word in lst_words_to_del]
                f_del.writelines(command_lines)

We have decided on the range of input sizes from 500 to 128000 at a two-fold increase since 500 is not too trivially small and the impact of increasing or decreasing input sizes according to a pattern makes it easy to analyze the time complexity. 

Since the empirical analysis outdoes the mathematical analysis in investigating the average-case efficiency, it makes more sense to generate multiple datasets per input size (e.g. ten 500-word datasets, ... , ten 128000-word datasets) and get the average time for an operation on the same set of datasets for each of the implementations. Therefore, a total of 90 datasets will be generated (9 input sizes * 10 datasets) as it helps lower the chances of measuring the running time of the worst-case scenario and producing biased results. 

For both Scenario 1 and 2, a dictionary of a given size (e.g. 500, ... , 128000) will be built per experiment and implementation. For Scenario 1, words, randomly selected, half as many as the number of words the dictionary carries will be added mostly in a bid to accurately reflect the presumed inefficiency of the add operation for the list-based dictionary. We will repeat the same process for Scenario 2 except the same proportion of words will be deleted from each dictionary (e.g. 250 randomly selected words deleted from the 500 words) for the same reason as above. For Scenario 3, Search / Auto-completion operations will be performed on dictionaries of 500, 8000, 128000 sizes as required by the assignment specification. 

### Measure the running time of Scenario 1

In [ ]:
import time

running_times = [[],[],[]] # list / hash / tst

s_time = time.time_ns()
# Iterate over 9 input sizes
for i in range(len(input_sizes)):
    running_times_per_input = [[],[],[]]
    # Iterate over 10 datasets per input size
    for j in range(num_datasets_per_input):
        data_filename = "generatedData/" + str(input_sizes[i]) + '-' + str(j) + '.txt'
        with open(data_filename, 'r') as f_dic: # 50-0.txt
            agents = [ListDictionary(), HashTableDictionary(), TernarySearchTreeDictionary()]
            words_frequencies_from_file = []
            # Build words_frequncies_from_file
            for line in f_dic:
                values = line.split()
                word = values[0]
                frequency = int(values[1])
                word_frequency = WordFrequency(word, frequency)  # each line contains a word and its frequency
                words_frequencies_from_file.append(word_frequency)
            # Iterate over each data structure
            for agent_idx, agent in enumerate(agents):
                agent.build_dictionary(words_frequencies_from_file)
                running_time = 0 

                command_filename = "generatedData/" + str(input_sizes[i]) + '-' + str(j) + '-add.in'
                with  open(command_filename, 'r') as f_add:
                    start_time = time.time_ns()
                    for line in f_add:
                        command_values = line.split()
                        command = command_values[0]
                        word = command_values[1]
                        frequency = int(command_values[2])
                        word_frequency = WordFrequency(word, frequency)
                        agent.add_word_frequency(word_frequency)
                    end_time = time.time_ns()
                    running_time = (end_time - start_time) / math.pow(10, 9)
                    running_times_per_input[agent_idx].append(running_time)
    print(i, running_times_per_input)
e_time = time.time_ns()
print(f'Time it took to run this block: {(e_time - s_time) / math.pow(10, 9)}')
        
        

        

        

0 [[0.013572, 0.0126, 0.012773, 0.011611, 0.010859, 0.012999, 0.014054, 0.011565, 0.010169, 0.009363], [0.013980000000000001, 0.012994, 0.013149, 0.011988, 0.011182000000000001, 0.013407, 0.01444, 0.011931, 0.010464999999999999, 0.009661], [0.01725, 0.016996, 0.015844, 0.014937, 0.014599, 0.016548, 0.017266, 0.014489, 0.012735999999999999, 0.011961]]
1 [[0.037151, 0.038343, 0.046405, 0.057251, 0.050442, 0.042477, 0.042587, 0.045037, 0.047437, 0.04663], [0.037823, 0.039303000000000005, 0.047068, 0.058056, 0.051246, 0.04331, 0.043376, 0.045774, 0.048167, 0.047338], [0.042881, 0.04480800000000001, 0.055358, 0.065328, 0.057639, 0.04866, 0.048621, 0.052415, 0.055441000000000004, 0.052633]]
2 [[0.157782, 0.175655, 0.179388, 0.147394, 0.152132, 0.148041, 0.141289, 0.144394, 0.147971, 0.145457], [0.158935, 0.176849, 0.18057399999999998, 0.148571, 0.15328799999999998, 0.14927, 0.142396, 0.145758, 0.149082, 0.146597], [0.171229, 0.19495600000000002, 0.19178299999999998, 0.16093400000000002, 0.16

In [33]:
lst = [[],[],[]]
lst[0].append(1)
lst[0].append(2)
lst

[[1, 2], [], []]